In [1]:
import sys
sys.path.append('/work/pikachu/utils/')
sys.path.append('/work/pikachu/third/')
sys.path.append('..')
from gezi.common import *
import gezi
from src import config
from src.config import *
from src import util
from src.dataset import Dataset
from src.eval import eval_seq, calc_score
gezi.init_flags()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/home/gezi/miniconda3/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/home/gezi/miniconda3/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/home/gezi/miniconda3/lib/python3.10/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
[

In [2]:
model = 'conformer-subsample.encoder_layers-16.encoder-squeezeformer.eval'
model_dir = f'../working/offline/29/0/{model}'

In [3]:
!ls {model_dir}

/bin/bash: /home/gezi/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
args.txt	       eval_step.txt  flags.txt		 log.txt
ckpt		       feats.npy      global.json	 outputs.npy
command.txt	       flags.json     history_flags.txt  script.txt
dataset-metadata.json  flags.pkl      log.html		 src


In [4]:
feats = gezi.load(f'{model_dir}/feats.npy')

In [5]:
feats.shape

(105120, 80, 192)

In [6]:
outputs = gezi.load(f'{model_dir}/outputs.npy')

In [7]:
outputs.shape

(105120, 80, 61)

In [8]:
preds = np.argmax(outputs, axis=-1)

In [9]:
preds.shape

(105120, 80)

In [18]:
embs = np.zeros([outputs.shape[-1], feats.shape[-1]])
embs.shape

(61, 192)

In [19]:
counts = [0] * outputs.shape[-1]

In [20]:
for pred, feat in tqdm(zip(preds, feats), total=len(preds)):
  for ch, f in zip(pred, feat):
    embs[ch] += f
    counts[ch] += 1

  0%|          | 0/105120 [00:00<?, ?it/s]

In [21]:
counts

[5033656,
 438092,
 2,
 1,
 2,
 130,
 27,
 2,
 8,
 13,
 4,
 7029,
 33,
 57021,
 25588,
 19020,
 24021,
 27781,
 25294,
 25767,
 26533,
 27476,
 25544,
 23734,
 24055,
 24232,
 2356,
 0,
 278,
 309,
 63,
 3,
 1083,
 207917,
 39361,
 86667,
 78059,
 305365,
 43117,
 54206,
 102636,
 166499,
 9302,
 26841,
 111688,
 69699,
 150396,
 198581,
 56732,
 4665,
 172583,
 162335,
 211000,
 73839,
 28972,
 57047,
 7579,
 48526,
 4296,
 17,
 92518]

In [24]:
for i, count in enumerate(counts):
  if count:
    embs[i] /= count

In [25]:
embs

array([[ 0.44820025,  0.69065138,  0.45368835, ...,  0.37246914,
        -0.45457715, -0.31162671],
       [ 0.49769263,  0.48278925,  0.50344379, ..., -0.16303883,
        -0.68316444, -0.08673915],
       [ 0.51613986,  0.14508873,  0.5298582 , ..., -0.07753445,
        -0.64062247,  0.04112925],
       ...,
       [ 0.18812206, -0.16804746,  0.54779373, ...,  0.02590111,
        -0.47409979, -0.29644366],
       [ 0.25106105,  0.38377634,  0.57509984, ...,  0.51392663,
        -0.78375396, -0.33757227],
       [ 0.08941885,  0.3215719 , -0.02799755, ...,  0.04354505,
        -0.86001586, -0.40195613]])

In [26]:
gezi.save(embs, f'{FLAGS.root}/embs.npy')